In [ ]:
%pip install numpy
%pip install nltk
%pip install scikit-learn

import nltk
import random
import string
import numpy as np
import io
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

# Lemmatizer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Tokenization from corpus file
def tokenize():
    with open("import_export_knowledge.txt", "r", encoding="utf-8") as file:
        corpus = file.read()
    sentence_tokens = nltk.sent_tokenize(corpus)
    word_tokens = nltk.word_tokenize(corpus)
    return sentence_tokens, word_tokens

# Preprocessing
def lemtokens(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

punct = dict((ord(i), None) for i in string.punctuation)

def lemmer(text):
    return lemtokens(nltk.word_tokenize(text.lower().translate(punct)))

# Greeting detection
greeting_inputs = ['hello', 'hi', 'hey', 'greetings']
greeting_responses = [
    'Hello, I am a chatbot. My name is Alan.',
    'Hello!',
    'Hi!',
    'Wassup!',
    'Hi there!'
]

def greeting(text):
    for token in text.split():
        if token.lower() in greeting_inputs:
            return random.choice(greeting_responses)

# Response generation
def respond(user_query):
    bot_response = ''
    sent_tokens, word_tokens = tokenize()
    sent_tokens.append(user_query)

    tfidf_obj = TfidfVectorizer(tokenizer=lemmer, stop_words='english')
    tfidf = tfidf_obj.fit_transform(sent_tokens)

    sim_values = cosine_similarity(tfidf[-1], tfidf)
    index = sim_values.argsort()[0][-2]
    flat = sim_values.flatten()
    flat.sort()
    required_tfidf = flat[-2]

    if required_tfidf == 0:
        bot_response += 'I cannot understand'
    else:
        bot_response += sent_tokens[index]

    return bot_response

# Main chatbot loop
print("Alan (Import/Export Chatbot)")
flag = 1
while flag:
    user_query = input("You: ").lower()
    if user_query == 'exit':
        flag = 0
        print("Alan: Bye, see you!")
    elif greeting(user_query) is not None:
        print("Alan:", greeting(user_query))
    else:
        print("Alan:", respond(user_query))